<a href="https://colab.research.google.com/github/snvssk/data240/blob/development/data240/tree/development/Data240_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

from datetime import datetime
from random import randint
from math import sqrt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, r2_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import os
import seaborn as sns
from glob import glob

import math
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
# Original Dataset
data = pd.read_csv("/content/drive/Shareddrives/DATA240 Team 3/weather_location_combine.csv")
data

,Unnamed: 0,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,...,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Location ID,Latitude,Longitude,Elevation
0,0,2020,1,1,0,0,-2.9,0,0,0,...,161.09,0.11,987,0.400,277.0,5.4,903523,41.89,-87.62,179
1,1,2020,1,1,0,30,-3.0,0,0,0,...,159.68,0.11,987,0.400,275.0,5.2,903523,41.89,-87.62,179
2,2,2020,1,1,1,0,-3.1,0,0,0,...,156.62,0.11,987,0.400,273.0,5.0,903523,41.89,-87.62,179
3,3,2020,1,1,1,30,-3.1,0,0,0,...,152.47,0.11,987,0.400,270.0,4.9,903523,41.89,-87.62,179
4,4,2020,1,1,2,0,-3.1,0,0,0,...,147.67,0.11,987,0.400,267.0,4.8,903523,41.89,-87.62,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788395,17515,2016,12,31,21,30,-2.0,0,0,0,...,139.45,0.07,1020,0.391,90.0,0.3,241108,47.61,-122.34,49
788396,17516,2016,12,31,22,0,-2.0,0,0,0,...,143.95,0.07,1020,0.392,122.1,0.4,241108,47.61,-122.34,49
788397,17517,2016,12,31,22,30,-2.0,0,0,0,...,148.03,0.07,1020,0.393,122.1,0.4,241108,47.61,-122.34,49
788398,17518,2016,12,31,23,0,-2.0,0,0,0,...,151.47,0.07,1020,0.395,144.7,0.4,241108,47.61,-122.34,49


# Data Pre-Processing

In [4]:
data['outcome'] = np.where(data['GHI'] > 350, 'high',(np.where(data['GHI'] < 200, 'low', 'medium')))
data

,Unnamed: 0,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,...,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Location ID,Latitude,Longitude,Elevation,outcome
0,0,2020,1,1,0,0,-2.9,0,0,0,...,0.11,987,0.400,277.0,5.4,903523,41.89,-87.62,179,low
1,1,2020,1,1,0,30,-3.0,0,0,0,...,0.11,987,0.400,275.0,5.2,903523,41.89,-87.62,179,low
2,2,2020,1,1,1,0,-3.1,0,0,0,...,0.11,987,0.400,273.0,5.0,903523,41.89,-87.62,179,low
3,3,2020,1,1,1,30,-3.1,0,0,0,...,0.11,987,0.400,270.0,4.9,903523,41.89,-87.62,179,low
4,4,2020,1,1,2,0,-3.1,0,0,0,...,0.11,987,0.400,267.0,4.8,903523,41.89,-87.62,179,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788395,17515,2016,12,31,21,30,-2.0,0,0,0,...,0.07,1020,0.391,90.0,0.3,241108,47.61,-122.34,49,low
788396,17516,2016,12,31,22,0,-2.0,0,0,0,...,0.07,1020,0.392,122.1,0.4,241108,47.61,-122.34,49,low
788397,17517,2016,12,31,22,30,-2.0,0,0,0,...,0.07,1020,0.393,122.1,0.4,241108,47.61,-122.34,49,low
788398,17518,2016,12,31,23,0,-2.0,0,0,0,...,0.07,1020,0.395,144.7,0.4,241108,47.61,-122.34,49,low


In [5]:
data.drop('Unnamed: 0', axis=1, inplace=True)  
data.drop('Latitude', axis=1, inplace=True)  
data.drop('Longitude', axis=1, inplace=True)  
data

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Location ID,Elevation,outcome
0,2020,1,1,0,0,-2.9,0,0,0,0,...,88.28,161.09,0.11,987,0.400,277.0,5.4,903523,179,low
1,2020,1,1,0,30,-3.0,0,0,0,4,...,88.94,159.68,0.11,987,0.400,275.0,5.2,903523,179,low
2,2020,1,1,1,0,-3.1,0,0,0,4,...,88.51,156.62,0.11,987,0.400,273.0,5.0,903523,179,low
3,2020,1,1,1,30,-3.1,0,0,0,4,...,88.51,152.47,0.11,987,0.400,270.0,4.9,903523,179,low
4,2020,1,1,2,0,-3.1,0,0,0,4,...,87.92,147.67,0.11,987,0.400,267.0,4.8,903523,179,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788395,2016,12,31,21,30,-2.0,0,0,0,0,...,68.00,139.45,0.07,1020,0.391,90.0,0.3,241108,49,low
788396,2016,12,31,22,0,-2.0,0,0,0,1,...,66.53,143.95,0.07,1020,0.392,122.1,0.4,241108,49,low
788397,2016,12,31,22,30,-2.0,0,0,0,0,...,66.52,148.03,0.07,1020,0.393,122.1,0.4,241108,49,low
788398,2016,12,31,23,0,-2.0,0,0,0,1,...,65.47,151.47,0.07,1020,0.395,144.7,0.4,241108,49,low


# Data Preperation

In [6]:
# To check number rows in each category
row_count = pd.DataFrame()
row_count['Before'] = data['outcome'].value_counts()
row_count

,Before
low,550439
high,170643
medium,67318


In [7]:
data = data.loc[(data['Hour'].isin([7, 10, 13, 16, 19])) & (data['Minute'] == 0)]
data

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Location ID,Elevation,outcome
14,2020,1,1,7,0,-3.5,0,0,0,4,...,92.20,93.80,0.11,987,0.400,236.0,5.0,903523,179,low
20,2020,1,1,10,0,0.0,59,875,358,2,...,79.80,70.01,0.11,986,0.300,205.0,6.7,903523,179,medium
26,2020,1,1,13,0,4.3,70,878,418,0,...,67.05,66.63,0.11,982,0.500,204.0,7.4,903523,179,high
32,2020,1,1,16,0,3.6,26,340,50,0,...,80.16,86.01,0.11,980,0.700,191.0,7.3,903523,179,low
38,2020,1,1,19,0,3.2,0,0,0,4,...,85.80,117.07,0.11,980,1.100,192.0,8.9,903523,179,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788366,2016,12,31,7,0,-1.0,0,0,0,4,...,88.51,99.30,0.07,1000,0.722,158.8,1.0,241108,49,low
788372,2016,12,31,10,0,1.0,54,642,202,4,...,100.00,76.67,0.07,1000,0.775,188.7,0.9,241108,49,medium
788378,2016,12,31,13,0,2.0,65,729,297,4,...,99.53,71.41,0.07,1000,1.055,186.5,1.2,241108,49,medium
788384,2016,12,31,16,0,0.0,17,379,36,1,...,73.67,87.15,0.07,1020,0.440,101.8,0.9,241108,49,low


In [8]:
data.isnull().sum()

Year                  0
Month                 0
Day                   0
Hour                  0
Minute                0
Temperature           0
Clearsky DHI          0
Clearsky DNI          0
Clearsky GHI          0
Cloud Type            0
Dew Point             0
DHI                   0
DNI                   0
GHI                   0
Relative Humidity     0
Solar Zenith Angle    0
Surface Albedo        0
Pressure              0
Precipitable Water    0
Wind Direction        0
Wind Speed            0
Location ID           0
Elevation             0
outcome               0
dtype: int64

# Data Transformation

In [9]:
X = data.drop(['GHI','outcome'], axis =1)
X

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,DNI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,Location ID,Elevation
14,2020,1,1,7,0,-3.5,0,0,0,4,...,0,92.20,93.80,0.11,987,0.400,236.0,5.0,903523,179
20,2020,1,1,10,0,0.0,59,875,358,2,...,414,79.80,70.01,0.11,986,0.300,205.0,6.7,903523,179
26,2020,1,1,13,0,4.3,70,878,418,0,...,878,67.05,66.63,0.11,982,0.500,204.0,7.4,903523,179
32,2020,1,1,16,0,3.6,26,340,50,0,...,203,80.16,86.01,0.11,980,0.700,191.0,7.3,903523,179
38,2020,1,1,19,0,3.2,0,0,0,4,...,0,85.80,117.07,0.11,980,1.100,192.0,8.9,903523,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788366,2016,12,31,7,0,-1.0,0,0,0,4,...,0,88.51,99.30,0.07,1000,0.722,158.8,1.0,241108,49
788372,2016,12,31,10,0,1.0,54,642,202,4,...,642,100.00,76.67,0.07,1000,0.775,188.7,0.9,241108,49
788378,2016,12,31,13,0,2.0,65,729,297,4,...,729,99.53,71.41,0.07,1000,1.055,186.5,1.2,241108,49
788384,2016,12,31,16,0,0.0,17,379,36,1,...,379,73.67,87.15,0.07,1020,0.440,101.8,0.9,241108,49


In [10]:
label_encoder = preprocessing.LabelEncoder()
data['outcome']= label_encoder.fit_transform(data['outcome']) 
y = data['outcome']
y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


14        1
20        2
26        0
32        1
38        1
         ..
788366    1
788372    2
788378    2
788384    1
788390    1
Name: outcome, Length: 82125, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [13]:
x_train

array([[-0.70710678, -0.44260244,  1.73704218, ..., -1.04071974,
        -1.18513389, -0.47702903],
       [-0.70710678,  0.42750469, -0.08191538, ..., -0.76022646,
         0.27862998, -0.0168116 ],
       [ 1.41421356,  1.0075761 , -1.21876386, ..., -1.18096638,
        -1.52190492, -0.52956527],
       ...,
       [ 0.        , -1.31270956, -1.33244871, ..., -0.9004731 ,
        -1.18513389, -0.47702903],
       [ 1.41421356, -0.15256673,  0.71387855, ..., -0.19923989,
         0.62033706, -0.2038406 ],
       [-0.70710678,  0.42750469,  1.39598764, ..., -1.22771526,
         1.76883479, -0.37405801]])

In [14]:
print(f' Shape of X_train : {x_train.shape}')
print(f' Shape of y_train : {y_train.shape}')
print(f' Shape of X_test : {x_test.shape}')
print(f' Shape of y_test : {y_test.shape}')

 Shape of X_train : (57487, 22)
 Shape of y_train : (57487,)
 Shape of X_test : (24638, 22)
 Shape of y_test : (24638,)


In [15]:
from tensorflow.keras.utils import to_categorical
model = keras.Sequential()
model.add(layers.InputLayer(input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Normalization(axis=None))
model.add(layers.Dense(300, activation="relu"))
model.add(layers.Dense(200, activation="relu"))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(25, activation="relu"))
model.add(layers.Dense(3, activation='softmax'))

In [16]:
model.compile(loss = 'categorical_crossentropy',     
   optimizer = 'adam', 
   metrics = ['accuracy'])

In [17]:
history = model.fit(x_train, to_categorical(y_train), epochs=100, batch_size=64)

Epoch 1/100
899/899 [==============================] - 4s 3ms/step - loss: 0.0919 - accuracy: 0.9625
Epoch 2/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0455 - accuracy: 0.9816
Epoch 3/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0373 - accuracy: 0.9841
Epoch 4/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0329 - accuracy: 0.9857
Epoch 5/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0307 - accuracy: 0.9868
Epoch 6/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0270 - accuracy: 0.9887
Epoch 7/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0263 - accuracy: 0.9891
Epoch 8/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0240 - accuracy: 0.9899
Epoch 9/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0221 - accuracy: 0.9906
Epoch 10/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0213 - accura

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 22)               3         
 n)                                                              
                                                                 
 dense (Dense)               (None, 300)               6900      
                                                                 
 dense_1 (Dense)             (None, 200)               60200     
                                                                 
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 25)                1275      
                                                        

In [19]:
y_pred=np.argmax(model.predict(x_test), axis=-1)

770/770 [==============================] - 1s 2ms/step


In [20]:
y_pred

array([1, 2, 0, ..., 1, 1, 0])

In [21]:
# Confustion Matrix and Accuracy Score 
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", cm)

#Classification Report
print(classification_report(y_test, y_pred))

# F1 Score
print("F1-Score", f1_score(y_test, y_pred,average='micro'))

print("Accuracy Score",accuracy_score(y_test, y_pred))

Confusion Matrix [[ 8568     0    19]
 [    0 12629    89]
 [   97    38  3198]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8587
           1       1.00      0.99      0.99     12718
           2       0.97      0.96      0.96      3333

    accuracy                           0.99     24638
   macro avg       0.98      0.98      0.98     24638
weighted avg       0.99      0.99      0.99     24638

F1-Score 0.99013718645994
Accuracy Score 0.99013718645994


# Model Training with selected feature

In [22]:
# Original Dataset
data = pd.read_csv("/content/drive/Shareddrives/DATA240 Team 3/weather_location_combine.csv")
data['outcome'] = np.where(data['GHI'] > 350, 'high',(np.where(data['GHI'] < 200, 'low', 'medium')))

In [23]:
data = data.loc[(data['Hour'].isin([7, 10, 13, 16, 19])) & (data['Minute'] == 0)]

In [24]:
X = data[['Clearsky GHI', 'Solar Zenith Angle', 'DHI', 'DNI', 'Clearsky DNI', 'Clearsky DHI', 'Cloud Type','Relative Humidity', 'Hour', 'Temperature', 'Month', 'Dew Point', 'Elevation']]


In [25]:
X

,Clearsky GHI,Solar Zenith Angle,DHI,DNI,Clearsky DNI,Clearsky DHI,Cloud Type,Relative Humidity,Hour,Temperature,Month,Dew Point,Elevation
14,0,93.80,0,0,0,0,4,92.20,7,-3.5,1,-4.6,179
20,358,70.01,112,414,875,59,2,79.80,10,0.0,1,-3.1,179
26,418,66.63,70,878,878,70,0,67.05,13,4.3,1,-1.3,179
32,50,86.01,28,203,340,26,0,80.16,16,3.6,1,0.5,179
38,0,117.07,0,0,0,0,4,85.80,19,3.2,1,1.1,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...
788366,0,99.30,0,0,0,0,4,88.51,7,-1.0,12,-2.0,49
788372,202,76.67,54,642,642,54,4,100.00,10,1.0,12,0.0,49
788378,297,71.41,65,729,729,65,4,99.53,13,2.0,12,0.0,49
788384,36,87.15,17,379,379,17,1,73.67,16,0.0,12,-5.0,49


In [26]:
label_encoder = preprocessing.LabelEncoder()
data['outcome']= label_encoder.fit_transform(data['outcome']) 
y = data['outcome']
print(y)

14        1
20        2
26        0
32        1
38        1
         ..
788366    1
788372    2
788378    2
788384    1
788390    1
Name: outcome, Length: 82125, dtype: int64


In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [29]:
print(f' Shape of X_train : {x_train.shape}')
print(f' Shape of y_train : {y_train.shape}')
print(f' Shape of X_test : {x_test.shape}')
print(f' Shape of y_test : {y_test.shape}')

 Shape of X_train : (57487, 13)
 Shape of y_train : (57487,)
 Shape of X_test : (24638, 13)
 Shape of y_test : (24638,)


In [30]:
from tensorflow.keras.utils import to_categorical
model = keras.Sequential()
model.add(layers.InputLayer(input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Normalization(axis=None))
model.add(layers.Dense(300, activation="relu"))
model.add(layers.Dense(200, activation="relu"))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(25, activation="relu"))
model.add(layers.Dense(3, activation='softmax'))

In [31]:
model.compile(loss = 'categorical_crossentropy',     
   optimizer = 'adam', 
   metrics = ['accuracy'])

In [32]:
history = model.fit(x_train, to_categorical(y_train), epochs=100, batch_size=64)

Epoch 1/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0800 - accuracy: 0.9673
Epoch 2/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0391 - accuracy: 0.9830
Epoch 3/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0332 - accuracy: 0.9863
Epoch 4/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0297 - accuracy: 0.9875
Epoch 5/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0282 - accuracy: 0.9878
Epoch 6/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0254 - accuracy: 0.9890
Epoch 7/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0242 - accuracy: 0.9895
Epoch 8/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0221 - accuracy: 0.9908
Epoch 9/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0216 - accuracy: 0.9905
Epoch 10/100
899/899 [==============================] - 3s 3ms/step - loss: 0.0209 - accura

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 13)               3         
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 300)               4200      
                                                                 
 dense_7 (Dense)             (None, 200)               60200     
                                                                 
 dense_8 (Dense)             (None, 100)               20100     
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dense_10 (Dense)            (None, 25)                1275      
                                                      

In [34]:
y_pred=np.argmax(model.predict(x_test), axis=-1)
y_pred

770/770 [==============================] - 1s 2ms/step


array([1, 2, 0, ..., 1, 1, 0])

In [35]:
# Confustion Matrix and Accuracy Score 
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", cm)

#Classification Report
print(classification_report(y_test, y_pred))

# F1 Score
print("F1-Score", f1_score(y_test, y_pred,average='micro'))

print("Accuracy Score",accuracy_score(y_test, y_pred))

Confusion Matrix [[ 8568     0    19]
 [    0 12692    26]
 [   40    30  3263]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8587
           1       1.00      1.00      1.00     12718
           2       0.99      0.98      0.98      3333

    accuracy                           1.00     24638
   macro avg       0.99      0.99      0.99     24638
weighted avg       1.00      1.00      1.00     24638

F1-Score 0.995332413345239
Accuracy Score 0.995332413345239
